In [ ]:
"""
@Author: Pavan Kalyan
@Date: 2022-12-18
@Last Modified by: Pavan Kalyan
@Last Modified time: 2022-12-20
@Title : Read and process covid .csv files and creating star schema and uploading it to s3
"""

In [14]:
import pandas as pd
import boto3
import json

In [16]:
s3 = boto3.client('s3')
try:
    response = s3.create_bucket(
        Bucket='cleanedcovid19data',
        CreateBucketConfiguration={
            'LocationConstraint': 'ap-south-1',
        },
    )
    print("Bucket created successfully")
    print(json.dumps(response, indent=4))
except Exception as e:
    print(e)

Bucket created successfully
{
    "ResponseMetadata": {
        "RequestId": "YD585BKVN1JE8PBG",
        "HostId": "lGUcEUQoPYFE269e2yp8CP8LX6QAsoMjtDyBOl4s6lwFhHvj2BW/PkOwQLUxsWinxrZaVt6KKVk=",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amz-id-2": "lGUcEUQoPYFE269e2yp8CP8LX6QAsoMjtDyBOl4s6lwFhHvj2BW/PkOwQLUxsWinxrZaVt6KKVk=",
            "x-amz-request-id": "YD585BKVN1JE8PBG",
            "date": "Fri, 16 Dec 2022 02:59:08 GMT",
            "location": "http://cleanedcovid19data.s3.amazonaws.com/",
            "server": "AmazonS3",
            "content-length": "0"
        },
        "RetryAttempts": 0
    },
    "Location": "http://cleanedcovid19data.s3.amazonaws.com/"
}


In [8]:
df = pd.read_csv('/home/ushkamalla/PycharmProjects/AWS/AWS-Project/pavan_projet/covid-19/owid-covid-latest.txt')
df.head(100)

,iso_code,continent,location,last_updated_date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2022-12-13,206743.0,41.0,58.857,7843.0,3.0,1.143,...,NaN,37.746,0.50,64.83,0.511,4.112877e+07,NaN,NaN,NaN,NaN
1,OWID_AFR,NaN,Africa,2022-12-13,12447396.0,696.0,1317.857,257346.0,5.0,14.571,...,NaN,NaN,NaN,NaN,NaN,1.426737e+09,NaN,NaN,NaN,NaN
2,ALB,Europe,Albania,2022-12-13,333567.0,32.0,16.000,3594.0,0.0,0.000,...,51.2,NaN,2.89,78.57,0.795,2.842318e+06,NaN,NaN,NaN,NaN
3,DZA,Africa,Algeria,2022-12-13,271149.0,2.0,3.857,6881.0,0.0,0.000,...,30.4,83.741,1.90,76.88,0.748,4.490323e+07,NaN,NaN,NaN,NaN
4,AND,Europe,Andorra,2022-12-13,47446.0,227.0,32.429,158.0,0.0,0.143,...,37.8,NaN,NaN,83.73,0.868,7.984300e+04,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,HUN,Europe,Hungary,2022-12-13,2171056.0,4704.0,672.000,48337.0,50.0,7.143,...,34.8,NaN,7.02,76.88,0.854,9.967304e+06,NaN,NaN,NaN,NaN
96,ISL,Europe,Iceland,2022-12-13,207101.0,0.0,0.000,219.0,0.0,0.000,...,15.2,NaN,2.91,82.99,0.949,3.729030e+05,NaN,NaN,NaN,NaN
97,IND,Asia,India,2022-12-13,44676470.0,152.0,185.429,530658.0,0.0,2.857,...,20.6,59.550,0.53,69.66,0.645,1.417173e+09,NaN,NaN,NaN,NaN
98,IDN,Asia,Indonesia,2022-12-13,6702132.0,2117.0,2278.714,160287.0,32.0,30.857,...,76.1,64.204,1.04,71.72,0.718,2.755013e+08,NaN,NaN,NaN,NaN


corona_cases

In [4]:
corona_cases=df[['iso_code','continent','location','last_updated_date','total_cases','new_cases',]]
corona_cases.head(50)

,iso_code,continent,location,last_updated_date,total_cases,new_cases
0,AFG,Asia,Afghanistan,2022-12-13,206743.0,41.0
1,OWID_AFR,NaN,Africa,2022-12-13,12447396.0,696.0
2,ALB,Europe,Albania,2022-12-13,333567.0,32.0
3,DZA,Africa,Algeria,2022-12-13,271149.0,2.0
4,AND,Europe,Andorra,2022-12-13,47446.0,227.0
5,AGO,Africa,Angola,2022-12-13,104808.0,0.0
6,AIA,North America,Anguilla,2022-12-13,3904.0,0.0
7,ATG,North America,Antigua and Barbuda,2022-12-13,9106.0,0.0
8,ARG,South America,Argentina,2022-12-13,9766975.0,27119.0
9,ARM,Asia,Armenia,2022-12-13,445881.0,0.0


In [17]:
corona_cases_csv = corona_cases.to_csv(index=False)
s3.put_object(Body=corona_cases_csv, Bucket="cleanedcovid19data", Key = 'corona_cases_data.csv')

{'ResponseMetadata': {'RequestId': 'VDCH2X5Y3SHH2VAH',
  'HostId': 'H+7xsAwgC5siqAHhmf+MsufIXWZ/GmMyvJnRFVa0BfMdjMwmw8ONey14Cq0eiXCrQx4nkikmUuEz+NxXIHCFwg==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'H+7xsAwgC5siqAHhmf+MsufIXWZ/GmMyvJnRFVa0BfMdjMwmw8ONey14Cq0eiXCrQx4nkikmUuEz+NxXIHCFwg==',
   'x-amz-request-id': 'VDCH2X5Y3SHH2VAH',
   'date': 'Fri, 16 Dec 2022 03:00:42 GMT',
   'etag': '"db662c17f0be746b92cf13906670ba71"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"db662c17f0be746b92cf13906670ba71"'}

dimDates

In [5]:
corona_date=df[['iso_code','last_updated_date',]]
corona_date.head(50)

,iso_code,last_updated_date
0,AFG,2022-12-13
1,OWID_AFR,2022-12-13
2,ALB,2022-12-13
3,DZA,2022-12-13
4,AND,2022-12-13
5,AGO,2022-12-13
6,AIA,2022-12-13
7,ATG,2022-12-13
8,ARG,2022-12-13
9,ARM,2022-12-13


In [ ]:
df3 = df.to_csv(index=False)
s3.put_object(Body=df3, Bucket="datafromdynamodbfilesupload", Key = 'date')

corona_deaths

In [19]:
corona_deaths=df[['iso_code','continent','location','last_updated_date','total_deaths','new_deaths','cardiovasc_death_rate','diabetes_prevalence','handwashing_facilities',]]
corona_deaths.head(50)

,iso_code,continent,location,last_updated_date,total_deaths,new_deaths,cardiovasc_death_rate,diabetes_prevalence,handwashing_facilities
0,AFG,Asia,Afghanistan,2022-12-13,7843.0,3.0,597.029,9.59,37.746
1,OWID_AFR,NaN,Africa,2022-12-13,257346.0,5.0,NaN,NaN,NaN
2,ALB,Europe,Albania,2022-12-13,3594.0,0.0,304.195,10.08,NaN
3,DZA,Africa,Algeria,2022-12-13,6881.0,0.0,278.364,6.73,83.741
4,AND,Europe,Andorra,2022-12-13,158.0,0.0,109.135,7.97,NaN
5,AGO,Africa,Angola,2022-12-13,1925.0,0.0,276.045,3.94,26.664
6,AIA,North America,Anguilla,2022-12-13,12.0,0.0,NaN,NaN,NaN
7,ATG,North America,Antigua and Barbuda,2022-12-13,146.0,0.0,191.511,13.17,NaN
8,ARG,South America,Argentina,2022-12-13,130041.0,0.0,191.032,5.50,NaN
9,ARM,Asia,Armenia,2022-12-13,8712.0,0.0,341.010,7.11,94.043


In [20]:
corona_deaths_csv = corona_deaths.to_csv(index=False)
s3.put_object(Body=corona_deaths_csv, Bucket="cleanedcovid19data", Key = 'corona_deaths.csv')

{'ResponseMetadata': {'RequestId': 'JB7HZDYJBRG63PTE',
  'HostId': '/6hOfI+b6pgVKYGyDjyiJDBK/qZsQovTxZ0jEB98VWvu0HpFMbpdwe0Ukjge1oULuOxUoSIcHW0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '/6hOfI+b6pgVKYGyDjyiJDBK/qZsQovTxZ0jEB98VWvu0HpFMbpdwe0Ukjge1oULuOxUoSIcHW0=',
   'x-amz-request-id': 'JB7HZDYJBRG63PTE',
   'date': 'Fri, 16 Dec 2022 03:03:20 GMT',
   'etag': '"0275277b16d5d49d1d0286b991548a66"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"0275277b16d5d49d1d0286b991548a66"'}

corona_hospital

In [22]:
corona_hospital=df[['iso_code','continent','location','last_updated_date','icu_patients','hosp_patients','weekly_icu_admissions','weekly_hosp_admissions','hospital_beds_per_thousand']]
corona_hospital.head(50)

,iso_code,continent,location,last_updated_date,icu_patients,hosp_patients,weekly_icu_admissions,weekly_hosp_admissions,hospital_beds_per_thousand
0,AFG,Asia,Afghanistan,2022-12-13,NaN,NaN,NaN,NaN,0.500
1,OWID_AFR,NaN,Africa,2022-12-13,NaN,NaN,NaN,NaN,NaN
2,ALB,Europe,Albania,2022-12-13,NaN,NaN,NaN,NaN,2.890
3,DZA,Africa,Algeria,2022-12-13,2.0,NaN,NaN,NaN,1.900
4,AND,Europe,Andorra,2022-12-13,NaN,NaN,NaN,NaN,NaN
5,AGO,Africa,Angola,2022-12-13,NaN,NaN,NaN,NaN,NaN
6,AIA,North America,Anguilla,2022-12-13,NaN,NaN,NaN,NaN,NaN
7,ATG,North America,Antigua and Barbuda,2022-12-13,NaN,NaN,NaN,NaN,3.800
8,ARG,South America,Argentina,2022-12-13,NaN,NaN,NaN,NaN,5.000
9,ARM,Asia,Armenia,2022-12-13,NaN,NaN,NaN,NaN,4.200


In [23]:
corona_hospital_csv = corona_hospital.to_csv(index=False)
s3.put_object(Body=corona_hospital_csv, Bucket="cleanedcovid19data", Key = 'corona_hospital.csv')

{'ResponseMetadata': {'RequestId': '8831R1YRNFJ96RFK',
  'HostId': 'H2RsSvjM12dTvQB2SPC69HMAoppo+Gg1wS+JoOFKLWYiKctdolpTvrhPHJyYrM1uCS5/eOK0lN0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'H2RsSvjM12dTvQB2SPC69HMAoppo+Gg1wS+JoOFKLWYiKctdolpTvrhPHJyYrM1uCS5/eOK0lN0=',
   'x-amz-request-id': '8831R1YRNFJ96RFK',
   'date': 'Fri, 16 Dec 2022 03:04:43 GMT',
   'etag': '"8530d5e1a436f9ee92523d569191172d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"8530d5e1a436f9ee92523d569191172d"'}

corona_vaccination

In [24]:
corona_vaccination=df[['iso_code','continent','location','last_updated_date','population','total_vaccinations','people_vaccinated','people_fully_vaccinated','total_boosters','new_vaccinations',]]
corona_vaccination.head(50)

,iso_code,continent,location,last_updated_date,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations
0,AFG,Asia,Afghanistan,2022-12-13,4.112877e+07,1.230490e+07,1.132091e+07,1.061563e+07,NaN,NaN
1,OWID_AFR,NaN,Africa,2022-12-13,1.426737e+09,7.031271e+08,4.584042e+08,3.725444e+08,6.000793e+07,8242.0
2,ALB,Europe,Albania,2022-12-13,2.842318e+06,NaN,NaN,NaN,NaN,NaN
3,DZA,Africa,Algeria,2022-12-13,4.490323e+07,NaN,NaN,NaN,NaN,NaN
4,AND,Europe,Andorra,2022-12-13,7.984300e+04,NaN,NaN,NaN,NaN,NaN
5,AGO,Africa,Angola,2022-12-13,3.558900e+07,2.346814e+07,1.488785e+07,8.193976e+06,1.368465e+06,NaN
6,AIA,North America,Anguilla,2022-12-13,1.587700e+04,2.444100e+04,1.085200e+04,1.037300e+04,3.216000e+03,NaN
7,ATG,North America,Antigua and Barbuda,2022-12-13,9.377200e+04,NaN,NaN,NaN,NaN,NaN
8,ARG,South America,Argentina,2022-12-13,4.551032e+07,1.111632e+08,4.138230e+07,3.788649e+07,3.221633e+07,89988.0
9,ARM,Asia,Armenia,2022-12-13,2.780472e+06,NaN,NaN,NaN,NaN,NaN


In [25]:
df = pd.read_csv('/home/ushkamalla/PycharmProjects/AWS/AWS-Project/pavan_projet/covid-19/vaccination_company.txt')
cleaned_data=df[['iso_code','vaccines']]    
cleaned_data.head(50)

,iso_code,vaccines
0,AFG,"CanSino, Covaxin, Johnson&Johnson, Moderna, Ox..."
1,ALB,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ..."
2,DZA,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac..."
3,AND,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
4,AGO,Oxford/AstraZeneca
5,AIA,"Oxford/AstraZeneca, Pfizer/BioNTech"
6,ATG,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
7,ARG,"CanSino, Moderna, Oxford/AstraZeneca, Pfizer/B..."
8,ARM,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, ..."
9,ABW,Pfizer/BioNTech


In [26]:
final_vaccination_data = pd.merge(corona_vaccination,cleaned_data, how='inner')
final_vaccination_data.head()

,iso_code,continent,location,last_updated_date,population,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,vaccines
0,AFG,Asia,Afghanistan,2022-12-13,41128772.0,12304901.0,11320910.0,10615628.0,NaN,NaN,"CanSino, Covaxin, Johnson&Johnson, Moderna, Ox..."
1,ALB,Europe,Albania,2022-12-13,2842318.0,NaN,NaN,NaN,NaN,NaN,"Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ..."
2,DZA,Africa,Algeria,2022-12-13,44903228.0,NaN,NaN,NaN,NaN,NaN,"Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac..."
3,AND,Europe,Andorra,2022-12-13,79843.0,NaN,NaN,NaN,NaN,NaN,"Moderna, Oxford/AstraZeneca, Pfizer/BioNTech"
4,AGO,Africa,Angola,2022-12-13,35588996.0,23468138.0,14887847.0,8193976.0,1368465.0,NaN,Oxford/AstraZeneca


In [27]:
final_vaccination_data_csv = final_vaccination_data.to_csv(index=False)
s3.put_object(Body=final_vaccination_data_csv, Bucket="cleanedcovid19data", Key = 'corona_vaccination.csv')

{'ResponseMetadata': {'RequestId': 'B26BJ3EDT758P5W3',
  'HostId': 'BC/aBG69p+1TwJq8AlGV4SpLAuuDFbPQSNiX9kEB/IAV+mAsN9wM4K5iHZhL0e8sdY6/I0AfPxnJgY3CCIJaYA==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'BC/aBG69p+1TwJq8AlGV4SpLAuuDFbPQSNiX9kEB/IAV+mAsN9wM4K5iHZhL0e8sdY6/I0AfPxnJgY3CCIJaYA==',
   'x-amz-request-id': 'B26BJ3EDT758P5W3',
   'date': 'Fri, 16 Dec 2022 03:05:47 GMT',
   'etag': '"ecc617b8e2ab74c3ae9ed9da29aed586"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"ecc617b8e2ab74c3ae9ed9da29aed586"'}